In [8]:
# prerequisite
# pip install -U imbalanced-learn

In [9]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split

from transformers.Rebalancer import Rebalancer

from utils.DataHelper import DataHelper
from utils.Common import Config
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.naive_bayes import GaussianNB

import pandas as pd
import numpy as np


In [10]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [11]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label)
new_df = pc.getFrame()

In [12]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [13]:
# pass feature to pipeline and convert it to numerical data
X = DataPipeline(Config.num_attribs,Config.cat_attribs).process(X)

In [14]:
X.isna().sum().sum()

0

In [15]:
Y.value_counts().tolist()

[14246, 2201]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=Config.test_size, stratify=Y)

In [17]:
from imblearn.over_sampling import SMOTE
smote_minority = SMOTE(n_jobs = -1, sampling_strategy = "minority")

X_train_sm, y_train_sm = smote_minority.fit_resample(X_train, y_train)


c:\Users\yvonn\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [11]:

# Create param grid
param_grid = {
              'var_smoothing': np.logspace(0,-9, num=100)
              } 

clf = GridSearchCV(
    estimator=GaussianNB(),
    scoring="accuracy",
    param_grid=param_grid,
    cv=10,
    refit=True,
    verbose=3
)

best_clf = clf.fit(X_train_sm, y_train_sm)

best_clf.best_score_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10] END ................var_smoothing=1.0;, score=0.578 total time=   0.1s
[CV 2/10] END ................var_smoothing=1.0;, score=0.581 total time=   0.0s
[CV 3/10] END ................var_smoothing=1.0;, score=0.601 total time=   0.0s
[CV 4/10] END ................var_smoothing=1.0;, score=0.572 total time=   0.0s
[CV 5/10] END ................var_smoothing=1.0;, score=0.580 total time=   0.0s
[CV 6/10] END ................var_smoothing=1.0;, score=0.594 total time=   0.0s
[CV 7/10] END ................var_smoothing=1.0;, score=0.596 total time=   0.0s
[CV 8/10] END ................var_smoothing=1.0;, score=0.593 total time=   0.0s
[CV 9/10] END ................var_smoothing=1.0;, score=0.586 total time=   0.0s
[CV 10/10] END ...............var_smoothing=1.0;, score=0.595 total time=   0.0s
[CV 1/10] END .var_smoothing=0.8111308307896871;, score=0.581 total time=   0.0s
[CV 2/10] END .var_smoothing=0.8111308307896

0.6594001678175254

In [12]:
best_model = best_clf.best_estimator_
best_model.score(X_test,y_test)

0.535258358662614

In [13]:
best_clf.best_params_
# {'var_smoothing': 0.012328467394420659}


{'var_smoothing': 0.012328467394420659}

In [19]:
import joblib
best_model = GaussianNB(var_smoothing= 0.012328467394420659)
best_model.fit(X_train_sm, y_train_sm)
joblib.dump(best_model,'../models/best_model_naivebayes.pkl')


['../models/best_model_naivebayes.pkl']